In [29]:
import csv
import pandas as pd
from sqlalchemy import create_engine, text

In [31]:
from mysql.connector import MySQLConnection, Error

In [33]:
# Making connection to SQL server
def make_connection():
    try:
        conn = MySQLConnection(host='IES-ADS-ClassDB.sjsu.edu',
                               database='querycrew_db',
                               user='querycrew_user',
                               password='Pomegranate_746')
        
        if conn.is_connected():
            print('Connected to the MySQL database!')
            
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [35]:
conn = make_connection()
cursor = conn.cursor()

Connected to the MySQL database!


We are going to make the view and use a join and display the contents of the view.

In [38]:
cursor.execute("DROP VIEW IF EXISTS vehicle_info")

view_sql = (
    """
    CREATE VIEW vehicle_info AS
    SELECT v.vin, v.make, v.model, vf.price_category 
    FROM vehicle v
    LEFT JOIN vehicle_transformed vf on v.vin = vf.vin;
    """
)

cursor.execute(view_sql)
conn.commit()

query_view = "SELECT * FROM vehicle_info"
cursor.execute(query_view)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df_view = pd.DataFrame(rows, columns=columns)

print("Created the view using a joins and displaying the contents.")
print()
print(df_view)

Created the view using a joins and displaying the contents.

                 vin        make    model price_category
0  1HGCM82633A123456      Toyota  Corolla            Low
1  1HGCM82633A654321       Honda    Civic            Mid
2  2FMDK49C59B123456        Ford   Escape            Low
3  3VW2K7AJ5JM123456  Volkswagen    Jetta            Low
4  4T1BE46K67U123456       Lexus   RX 350            Mid
5  5YJSA1E26HF123456       Tesla  Model S           High


We are now going to rewrite the view as a CTE and make it part of a SELECT statement. We will display the results of the query.

In [40]:
query_cte = """
    WITH vehicle_info_cte AS (
        SELECT v.vin, v.make, v.model, vf.price_category 
        FROM vehicle v
        LEFT JOIN vehicle_transformed vf ON v.vin = vf.vin
    )

    SELECT vi.make AS Make, vi.model as Model, vi.price_category
    FROM vehicle_info_cte vi
    ORDER BY vi.make;
"""

cursor.execute(query_cte)
rows = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]

df_cte = pd.DataFrame(rows, columns=columns)

# Display the results
print("Results:")
print(df_cte)


cursor.close()
conn.close()

print("\nConnection is Closed!")

Results:
         Make    Model price_category
0        Ford   Escape            Low
1       Honda    Civic            Mid
2       Lexus   RX 350            Mid
3       Tesla  Model S           High
4      Toyota  Corolla            Low
5  Volkswagen    Jetta            Low

Connection is Closed!
